## Research Project 3
```text
- Source: SEC
- Goal: Extract compensat
- Techniques: Regular Expressions, Probabilistic Context Free Grammar
- Tools: re, duckling```

In [49]:
# Standard library
import datetime

# Third-party
import gensim
import requests
from lxml import html

In [52]:
def get_ticker_headlines(ticker, date):
    str_date = date.strftime('%m%d%Y')
    url = 'https://www.reuters.com/finance/stocks/company-news/%s?date=%s' % (ticker, str_date)
    res = requests.get(url)
    tree = html.fromstring(res.content)
    headlines = []
    for div in tree.xpath('//div[@class="feature"]'):
        headline = list(div.itertext())[0]
        clean = [i for i in headline.split('-') if not i.isupper()]
        headlines.append('-'.join(clean))
    return headlines

In [55]:
total = set()
today = datetime.datetime(2018, 4, 1)
for day in range(30):
    date = today + datetime.timedelta(day)
    headlines = get_ticker_headlines('AAPL.O', date)
    for headline in headlines:
        if headline not in total:
            total.add(headline)
            print(str(date.date()) + ' - ' + headline)

2018-04-02 - Apple plans to replace Intel chips in Macs with its own -Bloomberg
2018-04-03 - Apple discloses gender pay gap at UK operations
2018-04-03 - Apple Reports Mean Pay Gap In UK Is 5 Pct Lower For Women
2018-04-03 - Tech, trade fears weighed on European shares
2018-04-03 - Tecnológicas, receios comércio pesam nas acções europeias
2018-04-04 - Apple Pay arrives in Brazil, partnering with Itaú Unibanco
2018-04-06 - Apple says repeal of U.S. EPA carbon plan would threaten investments
2018-04-09 - Apple says all its facilities now powered by clean energy
2018-04-09 - Apple Introduces iPhone 8 And iPhone 8 Plus Red Special Edition
2018-04-09 - Xiaomi pushes for smartphone component suppliers to invest in India
2018-04-10 - Apple adds Isaac Asimov sci-fi series to TV development list
2018-04-11 - Apple Music appoints new head, hits 48 mln subscribers
2018-04-16 - BlackRock's biggest stock-picking fund likes Facebook shares
2018-04-19 - Tobacco and tech drag on Wall St; yields boost 